# Implementing Dialog Managment

- **Referent:** Maurice Vogel
- **Datum:** 09.01.2020

## Outline
- Wiederholung von Ansätzen zum Dialogmanagment (DM)
  - "regel-basiertes" DM
  - "daten-basiertes" / statistisches DM<br><br>
- Kurzvorstellung der in der Demo verwendeten Frameworks
  - pandas
  - scikit-learn
  - Jupyter<br><br>
- Beispiel-Implementation des "regel-basierten" DM am Beispiel eines Pizzeria-Chatbots<br><br>
- Beispiel-Implementation des "daten-basierten" / statistisches  DM am Beispiel eines Pizzeria-Chatbots<br>

## Wiederholung von Ansätzen zum Dialogmanagment
- "regel-basiertes" DM
- "daten-basiertes" / statistisches DM

Was sind die wichtigsten Unterschiede zwischen diesen Ansätzen?

## Kurzvorstellung der in der Demo verwendeten Frameworks
Für die Demo im Buch verwendet McTear (2016) die beiden Frameworks Voice-XML und WEKA für Implementation der jeweiligen DM-Ansätze. Ich habe dies (vergleichsweise) ähnlich in Python umgesetzt und dafür hautpsächlich folgende Frameworks verwendet:

- [pandas](https://pandas.pydata.org/)
- [scikit-learn](https://scikit-learn.org/stable/)
- [nltk](https://www.nltk.org/)

Die Demo läuft hierbei innerhalb eines sog. [Jupyter](https://jupyter.org/)-Notebooks.

Allen Code für die Demo könnt ihr von meinem Git-Repository klonen (MIT-Lizenz). Für die Demo habe ich eine kleine Library für das DM mit den beiden o.g. Verfahren geschrieben.

Die Demo kann wie folgt importiert werden (auf der Verzeichnis-Ebene des Git Repos):

In [1]:
import lib 

Die einzelnen Pakete haben hierbei folgende Funktionen und sind objekt-orientiert implementiert:

In [2]:
# lib.data # Vorverarbeitung / Aufbereitung Trainingsdaten für statistisches DM
# lib.dm # Dialogmanagment (Hauptkomponente)
# lib.nlg # Sprachgenerierung auf Basis von Templates
# lib.nlu # Sprachverstehen, d.h. Informationsextraktion aus User Input über Pattern Matching
# lib.training # Training von statistischen DM-Modellen

## Demo (in Pycharm)

### Aufbereiten von Trainingsdaten

In [3]:
# Laden und Vorverarbeitung der Trainingsdaten
from lib.data import TrainDataLoader

tdl = TrainDataLoader("./data/Dialogs_Pizza.txt")
df = tdl.data

df.head()

,PREV_SYSTEM_ACTION,TypeOrder,NumberPizzas,TypesPizzas,SizesPizzas,TypesDoughs,Drinks,Acceptance,Rejection,NotUnderstood,SYSTEM_ACTION
0,1,0,1,0,0,0,0,0,0,0,4
1,4,0,1,1,0,0,0,0,0,0,5
7,4,0,1,1,0,0,0,0,0,0,5
11,1,0,1,0,0,0,0,0,0,0,4
12,4,0,1,2,0,0,0,0,0,0,10


### Training eines Modells



In [4]:
from lib.training import Trainer
import pandas as pd
data = TrainDataLoader("./data/Dialogs_Pizza.txt").data

# get feature matrix and target variable
X = data.drop(columns=["SYSTEM_ACTION"])
y = data["SYSTEM_ACTION"]

params_svm = {"kernel": ["rbf", "linear"], "gamma": [1e-3, 1e-4],"C": [1, 10, 100, 1000]}

eval_metrics = ['f1']

trainer = Trainer("svm")
trainer.train(X, y, params_svm, eval_metrics)

df_eval_results = pd.DataFrame(trainer.model.cv_results_)
df_eval_results.to_excel("./data/cv_results_svm.xlsx")

df_eval_results


# Tuning hyper-parameters for f1



d:\users\mvogel\pycharmprojects\word_embeddings_demo\venv\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
d:\users\mvogel\pycharmprojects\word_embeddings_demo\venv\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
d:\users\mvogel\pycharmprojects\word_embeddings_demo\venv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best parameters set found on development set:

{'C': 100, 'gamma': 0.001, 'kernel': 'linear'}

Grid scores on development set:

0.380 (+/-0.005) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.956 (+/-0.023) for {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
0.380 (+/-0.005) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.956 (+/-0.023) for {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
0.478 (+/-0.090) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.961 (+/-0.036) for {'C': 10, 'gamma': 0.001, 'kernel': 'linear'}
0.380 (+/-0.005) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.961 (+/-0.036) for {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
0.956 (+/-0.023) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.971 (+/-0.024) for {'C': 100, 'gamma': 0.001, 'kernel': 'linear'}
0.478 (+/-0.090) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.971 (+/-0.024) for {'C': 100, 'gamma': 0.0001, 'kernel': 'linear'}
0.956 (+/-0.023) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.971 (+/-0.02

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004639,0.000463,0.003657,0.000470,1,0.001,rbf,"{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}",0.376812,0.382353,0.382353,0.380488,0.002619,14
1,0.005004,0.000794,0.003410,0.000394,1,0.001,linear,"{'C': 1, 'gamma': 0.001, 'kernel': 'linear'}",0.942029,0.970588,0.955882,0.956098,0.011674,7
2,0.005208,0.000500,0.003001,0.000767,1,0.0001,rbf,"{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}",0.376812,0.382353,0.382353,0.380488,0.002619,14
3,0.005211,0.000985,0.001752,0.000356,1,0.0001,linear,"{'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}",0.942029,0.970588,0.955882,0.956098,0.011674,7
4,0.004994,0.000826,0.003660,0.000942,10,0.001,rbf,"{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}",0.536232,0.470588,0.426471,0.478049,0.045170,12
5,0.004655,0.000438,0.003655,0.000471,10,0.001,linear,"{'C': 10, 'gamma': 0.001, 'kernel': 'linear'}",0.942029,0.985294,0.955882,0.960976,0.018044,5
6,0.004869,0.000675,0.002161,0.000732,10,0.0001,rbf,"{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}",0.376812,0.382353,0.382353,0.380488,0.002619,14
7,0.008596,0.004414,0.002043,0.000073,10,0.0001,linear,"{'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}",0.942029,0.985294,0.955882,0.960976,0.018044,5
8,0.004975,0.001403,0.003669,0.001255,100,0.001,rbf,"{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}",0.942029,0.970588,0.955882,0.956098,0.011674,7
9,0.006333,0.000481,0.003410,0.000423,100,0.001,linear,"{'C': 100, 'gamma': 0.001, 'kernel': 'linear'}",0.956522,0.985294,0.970588,0.970732,0.011761,1


In [5]:
# model persistence
path_to_model = "./data/stat_model.joblib"
trainer.save(path_to_model)

trainer2 = Trainer().load(path_to_model)

trainer2.predict(X)

array([ 4,  5,  5,  4, 10,  5,  4,  5, 11,  4,  5,  9,  5,  5,  4, 10,  5,
        4,  5, 11,  4,  5,  4,  5,  4,  5,  5,  5,  5, 11, 10,  5,  5,  9,
        5,  4,  5,  5, 11, 10,  5,  5,  9,  5,  5, 10,  5, 10,  5, 11,  5,
        9,  5,  5, 10,  5, 11,  5,  5,  5,  3,  4,  5,  3,  9,  4,  5,  4,
        5,  3,  4, 10,  5,  3,  4,  5, 11,  3,  4,  5,  3,  4,  5,  4,  5,
        4, 10,  5,  3,  9,  4,  5,  3,  4,  5, 11,  3,  4,  5,  3,  4,  5,
        4,  5,  9,  4,  5,  9,  4, 10,  5,  9,  4, 10,  5, 11,  9,  4, 10,
        5, 11,  3,  4,  5,  4,  5,  3,  9,  4,  5,  3,  4, 10,  5,  3,  4,
        5, 11,  3,  4,  5,  3,  4,  5,  4,  5,  3,  9,  4,  5,  3,  9,  4,
        5, 11,  5,  9,  5, 10,  5,  5, 11,  5,  5,  5,  9,  5, 10,  5,  5,
        5, 11,  5,  5, 10,  5, 10,  5, 11,  5,  5, 10,  5,  5, 11, 11,  9,
       11, 14,  9, 10,  9,  9,  9,  9, 10,  9,  9, 14,  9,  9, 10, 14, 11,
       14,  4,  5,  4,  5,  4,  5,  4,  5,  4,  5,  4,  5,  9,  4,  5,  4,
        5,  4, 10,  5, 10